# Explore and cluster the neighborhoods in Toronto
## Scrape the Wiki page to get the list of PostalCode, Borough, and Neighborhood
### Only processing the cells that have an assigned borough. Ignoring the cells with a borough that is Not assigned
### cancating the neighborhoods based on postal code, seperated by ','
### print heads of dataframes
### Analyze neighborhoods details like we did in lab (Newyork, NY)

Let's start with importing the packages. we will use wiki pakage to scrape details from wiki and will use geopy to get latitude and longitude.. 


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge wikipedia
import wikipedia as wp

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  21.57 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  29.97 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  54.89 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  24.10 MB/s
vincent-0.4.4- 100% |###################

# Scraping Wiki to prepare Toronto dataframe

#### Tranform the data into a *pandas* dataframe

#### Borough and Neighbourhoods inforamtion extracted from Wiki, but there is no latitude and longitude information.  To get the latitude and longitude values of Borough and Neighbourhood of Toronto use geopy library and update each row of dataset

In [115]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]

#df.to_csv('beautifulsoup_pandas.csv',header=0,index=False)
df=df.rename(columns={0: 'PostalCode', 1:"Borough", 2: "Neighborhood"})
df.drop(df[df.Borough =="Not assigned"].index, inplace=True)

df.head(100)

,PostalCode,Borough,Neighborhood
0,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge


In [116]:
grouped= df.groupby(['PostalCode', 'Borough'], as_index=False, sort=True).apply(lambda group: ','.join(group['Neighborhood'])).reset_index()
grouped.rename(columns={0 : 'Neighborhood'}, inplace=True)
grouped.head(100)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [117]:
grouped.shape


(104, 3)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park

Update the Latitude and Longitude columns 

In [118]:


grouped.add('Latitude', fill_value=None)
grouped.add('Longitude', fill_value=None)
# Exclude the first row..
neighborhoods=grouped
#neighborhoods.dropna(thresh=2, inplace=True)

for key, data in neighborhoods.iterrows():
   # print(data['Neighbourhoods'])
   try: 
    if data['Neighborhood'] =='Not assigned':
        neighborhoods.loc[key,'Neighborhood'] = data['Borough']
    address = data['Neighborhood'] + ', ' + data['Borough'] + ',' + data['PostalCode']
       #print(address)
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    neighborhoods.loc[key,'Latitude'] = location.latitude
    neighborhoods.loc[key, 'Longitude'] = location.longitude
       #print( location.latitude, location.longitude)
   except:
        #print(address)
        pass
neighborhoods.head(100)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:16: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",NaN,NaN
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",NaN,NaN
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",NaN,NaN
3,M1G,Scarborough,Woburn,43.759824,-79.225291
4,M1H,Scarborough,Cedarbrae,NaN,NaN
5,M1J,Scarborough,Scarborough Village,NaN,NaN
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",NaN,NaN
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",NaN,NaN
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",NaN,NaN
9,M1N,Scarborough,"Birch Cliff,Cliffside West",NaN,NaN


In [119]:
#Remove all rows where no latitude and longitude... there may be a reason,geopy doesn't retrive latitude and longitude..
neighborhoods.dropna(inplace=True)
neighborhoods.head(100)

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 5 neighborhoods.


In [120]:
address = 'Toronto, CA'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto City are 43.653963, -79.387207.


#### Create a map of New York with neighborhoods superimposed on top.

In [121]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Folium is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in North York. So let's slice the original dataframe and create a new dataframe of the North York data.

In [122]:
NorthYork_data = neighborhoods[neighborhoods['Borough'] == 'North York'].reset_index(drop=True)
NorthYork_data.head(100)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M2N,North York,Willowdale South,43.769814,-79.41381


In [123]:
address = 'North York, Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of North York are 43.7709163, -79.4124102.


As we did with all of Toronto City, let's visualizat North York the neighborhoods in it.

In [124]:
# create map of Manhattan using latitude and longitude values
map_NorthYork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(NorthYork_data['Latitude'], NorthYork_data['Longitude'], NorthYork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NorthYork)  
    
map_NorthYork

 Foursquare API to explore the neighborhoods and segment them

In [125]:
#
CLIENT_ID = '3QMDP42DYXT0M5LCCXYJNY5B2WNPFZIKQEL1NAQA1FUFNYKC' # your Foursquare ID
CLIENT_SECRET = 'AD3MSB2U4MLVBBTK5YMNYBF2ULA5J4JSN3LO5NVWMQY4BEBD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3QMDP42DYXT0M5LCCXYJNY5B2WNPFZIKQEL1NAQA1FUFNYKC
CLIENT_SECRET:AD3MSB2U4MLVBBTK5YMNYBF2ULA5J4JSN3LO5NVWMQY4BEBD


In [126]:
NorthYork_data.loc[0, 'Neighborhood']
neighborhood_latitude = NorthYork_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = NorthYork_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = NorthYork_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Willowdale South are 43.7698142, -79.4138102.


Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.
First, let's create the GET request URL. Name your URL url.

In [127]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c3769934c1f67406a9644c8'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4aedfeadf964a52005d121e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1e0931735',
         'name': 'Coffee Shop',
         'pluralName': 'Coffee Shops',
         'primary': True,
         'shortName': 'Coffee Shop'}],
       'id': '4aedfeadf964a52005d121e3',
       'location': {'address': '5140 Yonge St',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Park Home Ave',
        'distance': 191,
        'formattedAddress': ['5140 Yonge St (at Park Home Ave)',
         'Toronto ON M2N 6L7',
         'Canada'],
        'labeledLat

In [128]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [129]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Starbucks,Coffee Shop,43.768192,-79.413021
1,Loblaws,Grocery Store,43.768648,-79.412597
2,Aroma Espresso Bar,Café,43.769449,-79.413081
3,The Keg,Steakhouse,43.766579,-79.412131
4,Cineplex Cinemas Empress Walk,Movie Theater,43.768625,-79.412613


In [130]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

51 venues were returned by Foursquare.


2. Explore Neighborhoods in North York

#### Let's create a function to repeat the same process to all the neighborhoods in North York


In [131]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [132]:
northyork_venues = getNearbyVenues(names=NorthYork_data['Neighborhood'],
                                   latitudes=NorthYork_data['Latitude'],
                                   longitudes=NorthYork_data['Longitude']
                                  )


Willowdale South


In [133]:
print(northyork_venues.shape)
northyork_venues.head()

(51, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Willowdale South,43.769814,-79.41381,Starbucks,43.768192,-79.413021,Coffee Shop
1,Willowdale South,43.769814,-79.41381,Loblaws,43.768648,-79.412597,Grocery Store
2,Willowdale South,43.769814,-79.41381,Aroma Espresso Bar,43.769449,-79.413081,Café
3,Willowdale South,43.769814,-79.41381,The Keg,43.766579,-79.412131,Steakhouse
4,Willowdale South,43.769814,-79.41381,Cineplex Cinemas Empress Walk,43.768625,-79.412613,Movie Theater


In [134]:
northyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Willowdale South,51,51,51,51,51,51


In [135]:
print('There are {} uniques categories.'.format(len(northyork_venues['Venue Category'].unique())))

There are 37 uniques categories.


## 3. Analyze Each Neighborhood

In [136]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot.head()

,Neighborhood,American Restaurant,Arts & Crafts Store,Beach,Bubble Tea Shop,Burrito Place,Café,Coffee Shop,Dessert Shop,Discount Store,Electronics Store,Fast Food Restaurant,Grocery Store,Hotel,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Lounge,Middle Eastern Restaurant,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Ramen Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Steakhouse,Supermarket,Sushi Restaurant,Theater,Vietnamese Restaurant,Wings Joint
0,Willowdale South,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Willowdale South,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Willowdale South,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Willowdale South,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,Willowdale South,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [137]:
northyork_onehot.shape

(51, 38)

In [138]:
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()
northyork_grouped

,Neighborhood,American Restaurant,Arts & Crafts Store,Beach,Bubble Tea Shop,Burrito Place,Café,Coffee Shop,Dessert Shop,Discount Store,Electronics Store,Fast Food Restaurant,Grocery Store,Hotel,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Lounge,Middle Eastern Restaurant,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Ramen Restaurant,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Steakhouse,Supermarket,Sushi Restaurant,Theater,Vietnamese Restaurant,Wings Joint
0,Willowdale South,0.019608,0.019608,0.019608,0.019608,0.019608,0.039216,0.078431,0.019608,0.019608,0.019608,0.039216,0.019608,0.019608,0.019608,0.019608,0.039216,0.039216,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608,0.039216,0.019608,0.019608,0.078431,0.058824,0.039216,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608,0.019608


In [139]:
northyork_grouped.shape

(1, 38)

#### Let's print each neighborhood along with the top 5 most common venues

In [140]:
num_top_venues = 5

for hood in northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Willowdale South----
                  venue  freq
0           Coffee Shop  0.08
1      Ramen Restaurant  0.08
2            Restaurant  0.06
3  Fast Food Restaurant  0.04
4        Sandwich Place  0.04




#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [141]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [142]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Willowdale South,Ramen Restaurant,Coffee Shop,Restaurant,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Sandwich Place,Korean Restaurant,Café,Burrito Place


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [143]:
# set number of clusters
kclusters = 1

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([0], dtype=int32)

In [144]:
#NorthYork_data.apply(lambda col: col.drop_duplicates().reset_index(drop=True))

northyork_merged = NorthYork_data
northyork_merged.shape
# add clustering labels
northyork_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

northyork_merged.head(50) # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2N,North York,Willowdale South,43.769814,-79.41381,0,Ramen Restaurant,Coffee Shop,Restaurant,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Sandwich Place,Korean Restaurant,Café,Burrito Place


In [145]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['Neighborhood'], northyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

In [146]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 0, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Ramen Restaurant,Coffee Shop,Restaurant,Fast Food Restaurant,Pizza Place,Japanese Restaurant,Sandwich Place,Korean Restaurant,Café,Burrito Place


In [105]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 1, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [106]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 2, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [107]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 3, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [108]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 4, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
